I used this code to generate sentences that I sent to our additional annotator. The idea was to replace annotations given to us by the bad annotator with Worker ID 'AJ9O2ZA0E8UDZ'.

In [18]:
import pandas as pd

annotations_df = pd.read_csv("annotations.csv")
original_corrected_df = annotations_df.iloc[:,[15,27, 28]]
original_corrected_df = original_corrected_df.rename(columns={"Input.orig": "Original", "Input.corr": "Corrected"})
original_corrected_df = original_corrected_df.loc[original_corrected_df['WorkerId'] == 'AJ9O2ZA0E8UDZ'].iloc[:,[1,2]]
original_corrected_df.to_csv("original_corrected.csv", index=False)
original_corrected_df

,Original,Corrected
19,And the fact that the queen herself remains ti...,And the fact that the Queen herself remains ti...
23,"This time it was easier to find help, a protec...","This time it was easier to find help, and a pr..."
27,People have their doubts deciphered by prophet...,People have their doubts deciphered by prophet...
35,Sa Dec is a symbolism of fairy land in Viet Na...,Sa Dec is symbolized as a fairy land in Viet N...
42,Finally yesterday I had interview in English a...,"Finally, yesterday I had an interview in Engli..."
...,...,...
1631,About the argument of how much it would help u...,About the argument of how much it would help u...
1637,Souza began making comic strips for newspapers...,Souza began making comic strips for newspapers...
1639,Yesterday I spent about 20 minutes to correct ...,Yesterday I spent about 20 minutes correcting...
1642,People tend to go to foreign capital company w...,People tend to go to foreign capital companies...


Below, I read in the file the annotator sent me back. They completed 99 out of 210 sentences that were sent, so I will be replacing those with the new annotations. As we can see from the output below, the classes are balanced with 50 'Yes' and 49 'No'.

In [121]:
import pandas as pd
oc_max_df = pd.read_csv("original_corrected_max.csv")
oc_max_df = oc_max_df.loc[:, ["Original", "Corrected", "label"]].iloc[:99]
oc_max_df.loc[oc_max_df['label'] == 'no','label'] = 'No'
oc_max_df.loc[oc_max_df['label']== 'no ','label'] = 'No'
oc_max_df.loc[oc_max_df['label'] == 'yes','label'] = 'Yes'
oc_max_df.loc[oc_max_df['label'] == 'yes ','label'] = 'Yes'
oc_max_df['label'].value_counts()

Yes    50
No     49
Name: label, dtype: int64

I read in the original dataset. As we can see, the annotations by the bad annotator have severe class imbalance - 98 'Yes' and 1 'No'. Replacing these by a more balanced set of annotations might improve agreement.

In [157]:
annotations_df = pd.read_csv("annotations.csv")
oc_max_df = oc_max_df.rename(columns={"Original": "Input.orig", "label": "Maxs_Label"})
first_100_orig = annotations_df.loc[annotations_df['WorkerId'] == 'AJ9O2ZA0E8UDZ'].iloc[:99]
first_100_orig['Answer.grammar-correction.label'].value_counts()

Yes    98
No      1
Name: Answer.grammar-correction.label, dtype: int64

I replace the 99 annotations:

In [166]:
first_100_orig["Maxs_Label"] = oc_max_df['Maxs_Label'].values
first_100_orig = first_100_orig.rename(columns={'Answer.grammar-correction.label':'old_label', 'Maxs_Label':'Answer.grammar-correction.label'})

for index, row in annotations_df.iterrows():
    if index > 939:
        break
    if row['WorkerId'] == 'AJ9O2ZA0E8UDZ':
        annotations_df.loc[index, "WorkerId"] = 'Max'
        annotations_df.loc[index, 'Answer.grammar-correction.label'] = first_100_orig.loc[index]['Answer.grammar-correction.label']


annotations_df.to_csv("annotations_w_max.csv", index=False)

To check that everything worked correctly:

In [167]:
annotations_df.loc[annotations_df['WorkerId']=='Max', 'Answer.grammar-correction.label'].value_counts()

Yes    50
No     49
Name: Answer.grammar-correction.label, dtype: int64

Now we test the interannotator agreement (code from Milestone 3):

In [170]:
import nltk
from nltk.metrics.agreement import AnnotationTask

def convert_to_tuples(annotations_list):
    # We need to have (Annotator, Sentence, Yes/No) triples
    triple_list = []
    for (index, (workers_list, label_list)) in enumerate(annotations_list):
        for i in range(len(label_list)):
            triple_list.append((workers_list[i], str(index),label_list[i]))
    
    return triple_list


annotations_df = pd.read_csv("annotations.csv")
hit_id_annotation_df = annotations_df.iloc[:,[0,15,29]]
hit_id_annotation_df = hit_id_annotation_df.rename(columns={"Answer.grammar-correction.label": "Label"})
hit_id_annotation_df = hit_id_annotation_df.groupby('HITId').agg(lambda x: x.tolist())
list_df = hit_id_annotation_df.values.tolist()

annotations_max_df = pd.read_csv("annotations_w_max.csv")
hit_id_annotation_df = annotations_max_df.iloc[:,[0,15,29]]
hit_id_annotation_df = hit_id_annotation_df.rename(columns={"Answer.grammar-correction.label": "Label"})
hit_id_annotation_df = hit_id_annotation_df.groupby('HITId').agg(lambda x: x.tolist())
list_df_max = hit_id_annotation_df.values.tolist()

triples = convert_to_tuples(list_df)

trips = []
for i, triple in enumerate(triples):
    triple = ("annotator"+str(i % 3), triple[1], triple[2])
    trips.append(triple)
    
triples = convert_to_tuples(list_df_max)

trips_max = []
for i, triple in enumerate(triples):
    triple = ("annotator"+str(i % 3), triple[1], triple[2])
    trips_max.append(triple)
    
annotation_task = AnnotationTask(trips)

print("Previous Pi agreement measure: ", annotation_task.pi()*100)

annotation_task_max = AnnotationTask(trips_max)

print("Improved Pi agreement measure: ", annotation_task_max.pi()*100)

Previous Pi agreement measure:  10.504105963138553
Improved Pi agreement measure:  13.134904126167681


As we can see, the annotator improved our agreement to 13.1% from 10.5%!